In [16]:
from IPython.display import display, Math

# Define the boolean expression
# clauses = [(1,3), (-1,2), (-2, 3)]
# clauses = [(1,2), (2,3), (1, 3)]
clauses = [(1, 2, 3), (1, 2, -3), (1, -2, 3), (1, -2, -3), (-1, 2, 3), (-1, 2, -3), (-1, -2, 3)]

# Get a list of variables
variables = set()
for clause in clauses:
    for variable in clause:
        variables.add(abs(variable))

print("List of variables:")
display(Math(', '.join(f'x{i}' for i in sorted(variables))))

# Print the boolean expression
latex_clauses = []
for clause in clauses:
    clause_literals = [f'\\overline{{x{abs(lit)}}}' if lit < 0 else f'x{abs(lit)}' for lit in clause]
    latex_clause = f'({(clause_literals[0])}'
    for clause_literal in clause_literals[1:]:
        latex_clause += f'\\lor {clause_literal}'
    latex_clause += f')'
    latex_clauses.append(latex_clause)
    
print("Given boolean expression:", end='')
display(Math(' \\land '.join(latex_clauses)))

List of variables:


<IPython.core.display.Math object>

Given boolean expression:

<IPython.core.display.Math object>